---

# 🧠 Socket Kütüphanesi Çalışmaları — Özet

Buraya kadar **socket kütüphanesi** üzerinde şunları yaptık:

- TCP bağlantısının nasıl kurulduğunu öğrendik.  
- `socket()`, `bind()`, `listen()`, `accept()`, `recv()`, `send()` fonksiyonlarını detaylı şekilde kullandık.  
- **Echo Server** yazarak istemci–sunucu arasında iki yönlü veri alışverişini test ettik.  
- **HTTP protokolünü** ham TCP üzerinden işleyip tarayıcıdan gelen istekleri çözdük.  
- **Mini Router Sistemi** oluşturarak `/`, `/hello`, `/data` gibi yolları yöneten kendi “mini framework” yapımızı geliştirdik.



🔧 **Şimdiki adım:**  
Bu yapıları genişletmek, yani kendi API çekirdeğimizi POST istekleri, dinamik route’lar ve çoklu istemci desteğiyle zenginleştirmek olacak.



📂 **Socket kütüphanesinin temel kullanımlarına tekrar bakmak isterseniz:**  

> ##### Kendi API'mız\Socket Kütüphanesi

---

# 🧩 Aşama 1 — HTTP Metod Desteği (GET & POST)

### 💡 Mantık

Artık sadece URL yoluna değil, hangi HTTP metoduyla geldiğine de bakacağız.

Yani “GET /hello” başka bir işlem, “POST /hello” başka bir işlem.


## ⚙️ Kod (asama_1.py)

In [1]:
import socket 

HOST = "127.0.0.1"
PORT = 8080
BUF = 4096

def handle_get_root():
    return 200, "text/html", "<h1>GET /</h1><p>Bu bir GET isteği.</p>"

def handle_post_root():
    return 200, "text/html", "<h1>POST /</h1><p>Bu bir POST isteği.</p>"

def handle_get_hello():
    return 200, "text/html", "<h1>Merhaba! 👋</h1><p>GET /hello isteği işlendi.</p>"


ROUTES = {
("GET" ,  "/") : handle_get_root,
("POST" , "/"): handle_post_root,
("GET" ,  "/") : handle_get_hello    
}

def make_response(status_code , content_type , body):
    status_texts = {
        200:"OK",
        404:"Not Found",
        405:"Method Not Allowed"
        }.get(status_code , "OK")

    response = f"""HTTP/1.1 {status_code} {status_texts}

Content-Type : {content_type} : charset = utf-8
Content-Length : {len(body)}

{body} """
    
    return response.encode("utf-8") 

In [ ]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen(1)
    print(f"🚀 Sunucu aktif: http://{HOST}:{PORT}")

    while True:
        client, addr = srv.accept()
        data = client.recv(BUF).decode("utf-8")

        if not data:
            client.close()
            continue

        # İlk satırı al: örn. "GET / HTTP/1.1"
        first_line = data.splitlines()[0]
        print(f"📩 İstek: {first_line}")

        try:
            method, path, _ = first_line.split()
        except ValueError:
            client.close()
            continue

        # Uygun route'u bul
        key = (method, path)
        if key in ROUTES:
            status, ctype, body = ROUTES[key]()
        else:
            # Aynı path varsa ama method yanlışsa 405
            allowed = [m for (m, p) in ROUTES if p == path]
            if allowed:
                status, ctype, body = 405, "text/html", f"<h1>405 Method Not Allowed</h1><p>İzin verilen: {', '.join(allowed)}</p>"
            else:
                status, ctype, body = 404, "text/html", "<h1>404 Not Found</h1>"

        # Yanıt gönder
        response = make_response(status, ctype, body)
        client.sendall(response)
        client.close()

#### 🔍 Sonuç

* Artık kendi mini API’m:

* Hem GET hem POST isteklerini ayırt edebiliyor,

* Hatalı metodları doğru HTTP koduyla döndürüyor.

## 🧩 1️⃣ Şu üç fonksiyonun (handle_*) görevi ne?

* Bunlar, gelen isteğe karşılık hangi cevabı vereceğini belirleyen fonksiyonlar.

* Yani FastAPI’de @app.get("/") gibi düşünebilirsin.
* Her biri belirli bir route (yol) ve HTTP metoduyla eşleşiyor.

### a) handle_get_root() ;

* Bu fonksiyon GET / isteği geldiğinde çağrılır.
>Yani tarayıcıda http://127.0.0.1:8080/ yazarsan bu çalışır.

Geriye şunları döndürür:

* 200 → durum kodu (başarılı)

* "text/html" → içerik türü

* body (HTML) → tarayıcıda gösterilecek içerik

### b) handle_post_root()

* Aynı yol (/) ama bu kez POST metoduyla çalışır.

Yani terminalden:

>curl -X POST http://127.0.0.1:8080/


yaparsan bu fonksiyon devreye girer.

### c) handle_get_hello()


* Bu da GET /hello isteğini işler.
>Yani tarayıcıda http://127.0.0.1:8080/hello yazarsan çalışır.

## 🧩 2️⃣ Peki ROUTES değişkeni ne?

* Bu bir harita (dictionary).
* Gelen isteğin türüne (metod ve yoluna) göre hangi fonksiyonun çalışacağını belirliyor.

```python
ROUTES = {
    ("GET", "/"): handle_get_root,
    ("POST", "/"): handle_post_root,
    ("GET", "/hello"): handle_get_hello,
}
```


* Burada anahtar (key) = (method, path)
* Yani "GET /" veya "POST /" şeklinde bir tanımlama.
* Değer (value) = o isteği işleyen fonksiyon.

## 🧩 `make_response()` Fonksiyonunun Mantığı

Bu fonksiyonun görevi, **sunucunun istemciye (tarayıcıya) göndereceği HTTP yanıtını formatlamak**.  

Yani "200 OK", "Content-Type", "Content-Length" gibi başlıkları düzgünce oluşturur ve mesaj gövdesiyle (body) birleştirir.



#### ⚙️ Fonksiyonun kendisi:
```python
def make_response(status_code, content_type, body):
    status_text = {
        200: "OK",
        404: "Not Found",
        405: "Method Not Allowed"
    }.get(status_code, "OK")

    response = f"""HTTP/1.1 {status_code} {status_text}
Content-Type: {content_type}; charset=utf-8
Content-Length: {len(body)}

{body}"""
    return response.encode("utf-8")


### 1️⃣ status_text

HTTP’nin durum kodlarını (200, 404, 405) yazıya çeviriyor.

* 200 → “OK”

* 404 → “Not Found”

* 405 → “Method Not Allowed”

Tarayıcıya “HTTP/1.1 200 OK” gibi düzgün bir başlık gönderebilmek için gerekiyor.

### 2️⃣ Content-Type

* Bu, yanıtın türünü belirtir.
* Tarayıcıya “bunu nasıl yorumlayacağını” söyler:

> text/html → HTML sayfası (tarayıcıda render edilir)

> application/json → JSON veri (API yanıtı)

> text/plain → düz metin

### 3️⃣ Content-Length

* Bu, gövdenin (body) byte cinsinden uzunluğunu belirtir.
* Tarayıcı bu değere bakarak kaç byte veri geleceğini bilir.

Örnek:
```http
Content-Length: 57
```


* Bu, “57 byte’lık veri geliyor” demektir.

### 4️⃣ body

* Body, sunucunun asıl göndermek istediği içeriktir.
>HTML veya JSON olabilir:
```html
<h1>Merhaba Dünya!</h1>
```

veya

```html
{"status": "ok", "message": "Hoş geldin"}
```

Yani make_response()’in parametreleri:

* status_code → 200, 404 gibi durum kodu

* content_type → “text/html” veya “application/json”

* body → gönderilecek içerik

## 🧩 Örnek:

* Fonksiyon çağrısı:

```python
response = make_response(200, "text/html", "<h1>Selam!</h1>")
```

* Tarayıcıya giden tam veri:
```bash
HTTP/1.1 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 14

<h1>Selam!</h1>

# 🧩 HTTP Router Mantığı — İstekten Yanıta Giden Yol 
```python
 try:
            method, path, _ = first_line.split()
        except ValueError:
            client.close()
            continue

        # Uygun route'u bul
        key = (method, path)
        if key in ROUTES:
            status, ctype, body = ROUTES[key]()
        else:
            # Aynı path varsa ama method yanlışsa 405
            allowed = [m for (m, p) in ROUTES if p == path]
            if allowed:
                status, ctype, body = 405, "text/html", f"<h1>405 Method Not Allowed</h1><p>İzin verilen: {', '.join(allowed)}</p>"
            else:
                status, ctype, body = 404, "text/html", "<h1>404 Not Found</h1>"

        # Yanıt gönder
        response = make_response(status, ctype, body)
        client.sendall(response)
        client.close()
```


## 🔹 1. İstek Satırının Çözülmesi
Tarayıcıdan gelen ilk satır genellikle şu şekildedir:

>GET /hello HTTP/1.1

Sunucu bu satırı üç parçaya ayırır:
- **Method:** İstek türü (örnek: GET, POST)
- **Path:** Hedef adres (örnek: / veya /hello)
- **Version:** Protokol sürümü (örnek: HTTP/1.1)

Bu bilgiler, isteğin “ne yapmak istediğini” anlamamızı sağlar.


## 🔹 2. Anahtar Oluşturma (method + path)
Metod ve yol birleştirilip bir **anahtar (tuple)** haline getirilir:

>("GET", "/hello")

Bu anahtar, `ROUTES` adlı sözlükte aranır.  
Bu sözlük, “hangi istek hangi fonksiyonu çalıştıracak” bilgisini tutar.  
Yani `ROUTES[("GET", "/hello")]` → `handle_get_hello()` fonksiyonunu gösterir.


## 🔹 3. Uygun Route’un Bulunması
Eğer bu anahtar `ROUTES` içinde varsa, ilgili fonksiyon çalıştırılır.  
Fonksiyon, şu üç bilgiyi döndürür:
- **Durum kodu (status):** 200, 404, 405 gibi
- **İçerik tipi (content-type):** text/html veya application/json
- **Body:** Tarayıcıya gösterilecek veya döndürülecek asıl içerik


## 🔹 4. Hatalı Durumların Yönetimi
Eğer gelen istek `ROUTES` içinde yoksa iki senaryo olur:

### a) Aynı path var ama metod farklı → 405  
Örneğin `/` sadece GET destekliyorsa ama sen POST gönderirsen,  
```bash
sunucu:  
405 Method Not Allowed
İzin verilen: GET
```
yanıtı gönderilir.



## 🔹 5. Yanıtın Oluşturulması ve Gönderilmesi
Fonksiyondan dönen bilgiler, `make_response()` fonksiyonuna gönderilir.  
Bu fonksiyon:
- HTTP başlıklarını oluşturur (status, content-type, content-length)
- Body ile birleştirir  
ve tam bir **HTTP yanıtı** haline getirir.

Sonrasında:
- `sendall()` → Yanıt istemciye (tarayıcıya) gönderilir.  
- `close()` → TCP bağlantısı kapatılır.


## 💡 Özet
Bu bölüm, sunucunun kalbidir:
1. İstek gelir.  
2. Metod ve path okunur.  
3. `ROUTES` içinde uygun fonksiyon bulunur.  
4. Fonksiyon yanıtı oluşturur.  
5. `make_response()` bunu HTTP biçiminde istemciye yollar.  

Kısacası: **İstek → Route → Yanıt.**



----

# 🔜 Aşama 2 Query String (URL parametreleri)

## 💡 Amaç

* Tarayıcıdan veya istemciden gelen URL’deki parametreleri okumak.

### Örnek:

>/hello?name=Hyso&age=21


Bu durumda:

* path → /hello

* query → {"name": "Ebrar", "age": "21"}

Bu bilgiyi çekip, fonksiyonlarda dinamik olarak kullanacağız.

## ⚙️ Kod (asama_2.py)

In [ ]:
# ==========================================================
# 🧩 AŞAMA 6 — Query String (URL Parametreleri)
# ==========================================================

import socket
from urllib.parse import urlsplit, parse_qs

HOST = "127.0.0.1"
PORT = 8080
BUF = 4096

# ---------------------------
# Yardımcı fonksiyonlar
# ---------------------------
def parse_request_line(request_line):
    """
    HTTP isteğinin ilk satırını (ör: 'GET /hello?name=Ebrar HTTP/1.1')
    parçalayarak method, path ve query verilerini döndürür.
    """
    method, target, _ = request_line.split()

    # urlsplit -> path ve query parçalarını ayırır
    parts = urlsplit(target)
    path = parts.path
    query = parse_qs(parts.query)  # {'name': ['Ebrar'], 'age': ['21']}

    # Tek değerli olanları sadeleştirelim:
    query = {k: v[0] if len(v) == 1 else v for k, v in query.items()}

    return method, path, query


def make_response(status, ctype, body):
    status_text = {200: "OK", 404: "Not Found"}.get(status, "OK")
    response = f"""HTTP/1.1 {status} {status_text}
Content-Type: {ctype}; charset=utf-8
Content-Length: {len(body)}

{body}"""
    return response.encode("utf-8")


# ---------------------------
# Route fonksiyonları
# ---------------------------
def handle_root(query):
    return 200, "text/html", "<h1>Ana Sayfa 🏠</h1><p>Hoş geldin!</p>"

def handle_hello(query):
    name = query.get("name", "Misafir")
    age = query.get("age")
    message = f"<h1>Merhaba, {name}! 👋</h1>"
    if age:
        message += f"<p>{age} yaşındasın!</p>"
    return 200, "text/html", message


# ---------------------------
# Router tablosu
# ---------------------------
ROUTES = {
    ( "GET", "/" ): handle_root,
    ( "GET", "/hello" ): handle_hello
}


# ---------------------------
# Sunucu
# ---------------------------
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen(1)
    print(f"🚀 Query destekli API aktif: http://{HOST}:{PORT}")

    while True:
        client, addr = srv.accept()
        data = client.recv(BUF).decode("utf-8")

        if not data:
            client.close()
            continue

        # İlk satırı al
        first_line = data.splitlines()[0]
        print(f"📩 {first_line}")

        try:
            method, path, query = parse_request_line(first_line)
        except ValueError:
            client.close()
            continue

        key = (method, path)
        if key in ROUTES:
            status, ctype, body = ROUTES[key](query)
        else:
            status, ctype, body = 404, "text/html", "<h1>404 Not Found</h1>"

        response = make_response(status, ctype, body)
        client.sendall(response)
        client.close()


🚀 Query destekli API aktif: http://127.0.0.1:8080
📩 GET / HTTP/1.1
📩 GET /favicon.ico HTTP/1.1
📩 GET /hello?name=Ebrar HTTP/1.1
📩 GET /hello?name=Ebrar&age=21 HTTP/1.1


### 🧠 Ne Öğrendik?

* URL içindeki ? işaretinden sonra gelen parametreleri parse etmeyi öğrendik.

>urllib.parse.urlsplit() ve parse_qs() ile:

* path → /hello

* query → {'name': 'Ebrar', 'age': '21'}
biçiminde ayırmayı başardık.

* Artık API’miz dinamik cevap verebiliyor.

## 🧩 `parse_request_line()` Fonksiyonunun Mantığı

Bu fonksiyon, HTTP isteğinin **ilk satırını** alır ve onu **üç parçaya ayırır:**
1. `method` → İstek türü (örnek: GET veya POST)  
2. `path` → URL yolunun kendisi (örnek: /hello)  
3. `query` → URL içindeki parametreler (örnek: {"name": "Ebrar", "age": "21"})  


### 💡 1️⃣ Girdi Nedir?

Bir HTTP isteği şu şekilde başlar:


* GET /hello?name=Hyso&age=21 HTTP/1.1

Bu satır, “**method path protokol**” formatındadır.

Fonksiyonun görevi bu satırı parçalamaktır.



### ⚙️ 2️⃣ `method, target, _ = request_line.split()`

Bu satır, gelen metni üçe böler:
- `method` → `"GET"`  
- `target` → `"/hello?name=Hyso&age=21"`  
- `_` → `"HTTP/1.1"` (protokol kısmı; şu an gerekmediği için `_` ile yok sayılır)

Yani bu noktada şunları elde ettik:
```python
method = "GET"
target = "/hello?name=Hyso&age=21"


### 🧩 3️⃣ urlsplit(target)

* Python’un urllib.parse modülünden gelen bir fonksiyondur.

URL’yi parçalara ayırır:

```python
parts = urlsplit("/hello?name=Hyso&age=21")
```

Sonuç:

* parts.path  -> "/hello"
* parts.query -> "name=Hyso&age=21"

Bu sayede path ve query kısımlarını ayrı ayrı ele alabiliriz

### 🔍 4️⃣ parse_qs(parts.query)

* parse_qs() fonksiyonu, query kısmını sözlük (dict) haline getirir:

> parse_qs("name=Hyso&age=21")


Sonuç:

> {"name": ["Hyso"], "age": ["21"]}


* Yani her değer liste olarak gelir; çünkü aynı parametre birden fazla olabilir:
```ruby

?tag=python&tag=ai → {'tag': ['python', 'ai']}


### 🔧 5️⃣ Sözlüğü sadeleştirme

* Aşağıdaki satır, tek elemanlı listeleri sadeleştirir:

>query = {k: v[0] if len(v) == 1 else v for k, v in query.items()}


Böylece:

 > {"name": ["Ebrar"], "age": ["21"]}


yerine:

> {"name": "Ebrar", "age": "21"}


elde ederiz.
* Ama bir parametrenin birden fazla değeri varsa, o liste olarak kalır.

### 🧠 6️⃣ Dönüş Değeri

* Fonksiyon üç bilgiyi döndürür:

>return method, path, query


Örnek çıktı:
```python
("GET", "/hello", {"name": "Ebrar", "age": "21"})

## 💬 Özet

| Değişken | Anlamı            | Örnek                          |
| -------- | ----------------- | ------------------------------ |
| `method` | HTTP isteği türü  | "GET"                          |
| `path`   | İstenen yol       | "/hello"                       |
| `query`  | URL parametreleri | {"name": "Ebrar", "age": "21"} |


* Bu fonksiyon sayesinde, tarayıcıdan gelen URL’yi okunabilir ve kullanılabilir hale getiriyoruz.
* FastAPI’nin veya Flask’ın request.args yapısının ilkel versiyonunu aslında burada kendimiz yazdık. 

----
# 🔜 Aşama 3 — JSON Body (POST Verisi İşleme)
### 🎯 Amaç

* Artık tarayıcı ya da Postman bize JSON gövde (body) gönderecek,biz de o veriyi okuyup işleyip yine JSON olarak döndüreceğiz.

### Örnek:

```json
İstek (POST):

{
  "a": 10,
  "b": 5
}


Yanıt (JSON):

{
  "a": 10,
  "b": 5,
  "sum": 15
}

### 💡 Bu aşamada öğreneceğiz:

* HTTP isteğinin body (gövde) kısmını okumayı (recv() ile Content-Length kadar)

* Gelen gövdeyi JSON olarak parse etmeyi (json.loads())

* application/json tipinde yanıt döndürmeyi

* Hatalı JSON geldiğinde 400 “Bad Request” döndürmeyi

* Küçük bir örnek endpoint: /sum → toplama yapan API

## ⚙️ Şu anda elimizde ne var?

#### ✅ 1. Metod farkını anlıyoruz:,


GET ve POST ayrımını yapabiliyoruz.

#### ✅ 2. Query string okuyabiliyoruz:

/hello?name=Hyso → {"name": "Hyso"}

#### ✅ 3. HTML yanıtı oluşturabiliyoruz:

make_response() ile düzgün HTTP formatında yanıt dönüyoruz.

## 🚀 Şimdi neleri ekleyeceğiz?
#### 🔹 1️⃣ HTTP Body (Gövde) Okuma

Bir POST isteği geldiğinde veri şuraya gelir:

```css
Content-Length: 28
Content-Type: application/json

{"a": 10, "b": 5}



### Yani:

Başlıklar (headers) yukarıda,

JSON veri (body) aşağıda.

Biz recv() ile başlıkları okuduktan sonra, Content-Length değerine bakıp o kadar byte okumalıyız.

### Eklenecek işlem:

```python
content_length = int(headers["content-length"])
body = client.recv(content_length)


### 🔹 2️⃣ JSON Veriyi Parse Etme

İstekten gelen body aslında bir string:

> '{"a": 10, "b": 5}'


Bunu Python nesnesine çevirmek için:

```python
import json
data = json.loads(body)
```


Artık data bir dict olur:

> {"a": 10, "b": 5}


### 🔹 3️⃣ Veriyi İşleme (örnek işlem)

Test için basit bir matematik işlemi yapalım:
```body
result = data["a"] + data["b"]
```


ve bunu JSON olarak geri gönderelim:

```body
esponse_body = json.dumps({"sum": result})

### 🔹 4️⃣ Yanıtı JSON Formatında Döndürme

Artık HTML yerine JSON tipinde yanıt döneceğiz:

**Content-Type: application/json**

```bash
Örnek yanıt:

{
  "a": 10,
  "b": 5,
  "sum": 15
}

### 🔹 5️⃣ Hata Yönetimi (Bad Request)

* Eğer kullanıcı yanlış JSON gönderirse, sunucu çökmemeli.
Bu durumda try / except ile kontrol edeceğiz:

```python
try:
    data = json.loads(body)
except json.JSONDecodeError:
    return 400, "application/json", json.dumps({"error": "Geçersiz JSON"})

### 🔹 6️⃣ Route Tablosuna Yeni Bir POST Endpoint

#### Yeni endpoint’imiz:

**POST /sum**


* Fonksiyon: handle_post_sum()
* Görev: JSON içindeki “a” ve “b” değerlerini toplayıp sonucu döndürmek.

## 🔧 Özetle Eklenecekler

| Özellik                  | Açıklama                                     |
| ------------------------ | -------------------------------------------- |
| **Content-Length okuma** | Gövdenin uzunluğunu belirleme                |
| **Body okuma**           | `recv()` ile JSON içeriği alma               |
| **JSON parse**           | `json.loads()` ile Python dict’e çevirme     |
| **Veri işleme**          | Gönderilen verilerle işlem yapma             |
| **JSON response**        | `json.dumps()` ile JSON yanıt döndürme       |
| **Hata yönetimi**        | Bozuk JSON veya eksik alanlarda 400 döndürme |


## **Kod yapısına bakalım** ( asama_3.py )

In [5]:
import socket , json

HOST="127.0.0.1"
PORT = 8082
BUF = 4096

def make_response(status, content_type, body):
    """HTTP yanıtını formatlar."""
    status_text = {200: "OK", 400: "Bad Request", 404: "Not Found"}.get(status, "OK")
    response = f"""HTTP/1.1 {status} {status_text}
Content-Type: {content_type}; charset=utf-8
Content-Length: {len(body)}

{body}"""
    return response.encode("utf-8")

In [6]:
def parse_request(data):
    headers_part, _, body = data.partition("\r\n\r\n")
    lines = headers_part.split("\r\n")
    method , path , _ = lines[0].split()

    headers = {}

    for line in lines[1:]:
        if ":" in line:
            key , value = line.strip(":" , 1 )
            headers[key.strip().lower()] = value.strip()
        
    return method , path , headers , body

In [7]:
def handle_get_root():
    """Basit GET endpoint."""
    body = "<h1>Mini API’ye hoş geldin 🧠</h1><p>POST /sum endpoint’ini dene!</p>"
    return 200, "text/html", body

In [8]:
def handle_post_sum(headers, body):
    """POST /sum → JSON verisini alır, toplar ve sonucu döner."""
    # Content-Type kontrolü
    if headers.get("content-type", "") != "application/json":
        err = json.dumps({"error": "Content-Type application/json olmalı"}, ensure_ascii=False)
        return 400, "application/json", err

    # Gövdeyi JSON’a çevir
    try:
        data = json.loads(body)
    except json.JSONDecodeError:
        err = json.dumps({"error": "Geçersiz JSON"}, ensure_ascii=False)
        return 400, "application/json", err

    # a ve b anahtarlarını kontrol et
    if not all(k in data for k in ("a", "b")):
        err = json.dumps({"error": "JSON içinde 'a' ve 'b' anahtarları olmalı"}, ensure_ascii=False)
        return 400, "application/json", err

    # İşlemi yap
    try:
        a, b = float(data["a"]), float(data["b"])
        toplam = a + b
    except Exception:
        err = json.dumps({"error": "a ve b sayısal olmalı"}, ensure_ascii=False)
        return 400, "application/json", err

    # Sonucu döndür
    result = json.dumps({"a": a, "b": b, "sum": toplam}, ensure_ascii=False)
    return 200, "application/json", result

In [ ]:
ROUTES = {
    ("GET", "/"): handle_get_root,
    ("POST", "/sum"): handle_post_sum,
}

In [13]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen(1)
    print(f"🚀 JSON destekli mini API aktif: http://{HOST}:{PORT}")

    while True:
        conn, addr = srv.accept()
        data = b""
        while True:
            chunk = conn.recv(BUF)
            if not chunk:
                break
            data += chunk
            if b"\r\n\r\n" in data:
                # Başlıkları aldıktan sonra Content-Length kontrol et
                head, sep, rest = data.partition(b"\r\n\r\n")
                headers_text = head.decode("utf-8", errors="ignore")
                if "Content-Length" in headers_text:
                    for line in headers_text.split("\r\n"):
                        if line.lower().startswith("content-length:"):
                            length = int(line.split(":")[1].strip())
                            break
                    # Gövde tamamlanana kadar okumaya devam et
                    while len(rest) < length:
                        rest += conn.recv(BUF)
                data = head + sep + rest
                break

        # İstek çözümleme
        raw = data.decode("utf-8", errors="ignore")
        method, path, headers, body = parse_request(raw)

        # Uygun route seçimi
        key = (method, path)
        if key == ("POST", "/sum"):
            status, ctype, response_body = handle_post_sum(headers, body)
        elif key == ("GET", "/"):
            status, ctype, response_body = handle_get_root()
        else:
            status, ctype, response_body = 404, "text/html", "<h1>404 Not Found</h1>"

        response = make_response(status, ctype, response_body)
        conn.sendall(response)
        conn.close()

API dinleniyor : http://127.0.0.1:8082


---
# SORU KISIMLARI VE KOD AÇIKLAMALARI AŞAĞIDADIR
---


```python
def parse_request(data):
    headers_part, _, body = data.partition("\r\n\r\n")
    lines = headers_part.split("\r\n")
    method , path , _ = lines[0].split()

    headers = {}

    for line in lines[1:]:
        if ":" in line:
            key , value = line.strip(":" , 1 )
            headers[key.strip().lower()] = value.strip()
        
    return method , path , headers , body
```


## Soru 1-) Bu fonksyonda bulunan işlemler nedir? Partition ve split içerisindeki "r" ve "n"  konfigürasyonları neden yapıldı? Strip içerisindeki ":" ifadesi ne işe yarıyor ?

### 🧠 1️⃣ headers_part, _, body = data.partition("\r\n\r\n")

* partition(), bir metni belirli bir ayırıcıya göre üç parçaya böler:

Ayırıcının öncesi

Ayırıcının kendisi (_ değişkenine atanıyor çünkü işimize yaramıyor)

Ayırıcının sonrası

Burada ayırıcı "\r\n\r\n".

### 🧩 Peki bu \r\n\r\n ne?

HTTP isteklerinin yapısı şu şekildedir:
```BASH
GET /sum HTTP/1.1\r\n
Host: 127.0.0.1:8080\r\n
Content-Type: application/json\r\n
Content-Length: 18\r\n
\r\n
{"a": 10, "b": 5}
```


* \r\n → satır sonu karakteridir (\r = carriage return, \n = newline).

* HTTP standardında her satır bu karakterle biter.

* Başlıklar (headers) bittikten sonra bir boş satır vardır.
* İşte o boş satır iki kez \r\n’dir → \r\n\r\n.

Yani:

> headers_part → başlık satırları (GET /sum HTTP/1.1, Content-Type: ...)

> body → JSON veya form verisi ({"a": 10, "b": 5})

### 🧠 2️⃣ lines = headers_part.split("\r\n")

* Bu, başlık kısmındaki satırları tek tek ayırır:
```bash

[
  "GET /sum HTTP/1.1",
  "Host: 127.0.0.1:8080",
  "Content-Type: application/json",
  "Content-Length: 18"
]
```


* Artık her satırı sırayla işleyebiliriz.

### 🧠 3️⃣ method, path, _ = lines[0].split()

* İlk satır GET /sum HTTP/1.1 olduğu için split() ile 3 parçaya ayrılır:

>method → "GET"

>path → "/sum"

>_ → "HTTP/1.1" (şimdilik ilgilenmiyoruz)

### 🧠 4️⃣ Header’ları sözlüğe çevirme

Her header satırı şu formattadır:

* Anahtar: Değer


Örnek:

> Content-Type: application/json


* Bu satırı ikiye ayırmak için:

>key, value = line.split(":", 1)


* Yani sadece ilk iki noktadan bölme yapar (bazı header’larda değer içinde de : olabilir).

### 🧩 strip() ne işe yarıyor?

* strip() metnin başındaki ve sonundaki boşlukları veya belirttiğin karakterleri temizler.

Burada:

> key.strip().lower()


→ Anahtarın başındaki veya sonundaki boşlukları atar, küçük harfe çevirir.

> value.strip()


→ Değerin başındaki ve sonundaki boşlukları atar.

### 🧩 5️⃣ Dönüş Değeri

| Değer     | İçerik           | Örnek                                                          |
| --------- | ---------------- | -------------------------------------------------------------- |
| `method`  | HTTP isteği türü | `"POST"`                                                       |
| `path`    | İstenen adres    | `"/sum"`                                                       |
| `headers` | Tüm başlıklar    | `{"content-type": "application/json", "content-length": "18"}` |
| `body`    | Gönderilen veri  | `'{"a": 10, "b": 5}'`                                          |


### 🧭 Özet
| İşlem                   | Amaç                             | Örnek Sonuç                          |
| ----------------------- | -------------------------------- | ------------------------------------ |
| `partition("\r\n\r\n")` | Başlıklar ve gövdeyi ayırır      | headers vs body                      |
| `split("\r\n")`         | Satırları liste haline getirir   | ["GET / HTTP/1.1", "Host: ..."]      |
| `split(":", 1)`         | Header anahtarı ve değeri ayırır | ("Content-Type", "application/json") |
| `strip()`               | Baş/son boşlukları temizler      | `"content-type"`                     |
| `lower()`               | Anahtarları küçük harfe çevirir  | `"content-type"`                     |
| `body`                  | JSON veya form verisi            | `'{"a": 10, "b": 5}'`                |



### Yani bu fonksiyon, gelen karmaşık HTTP metnini alıp şuna çeviriyor:
```bash
{
  "method": "POST",
  "path": "/sum",
  "headers": {"content-type": "application/json", "content-length": "18"},
  "body": '{"a": 10, "b": 5}'
}


--------

----------
```python
def handle_post_sum(headers, body):
    """POST /sum → JSON verisini alır, toplar ve sonucu döner."""
    # Content-Type kontrolü
    if headers.get("content-type", "") != "application/json":
        err = json.dumps({"error": "Content-Type application/json olmalı"}, ensure_ascii=False)
        return 400, "application/json", err

    # Gövdeyi JSON’a çevir
    try:
        data = json.loads(body)
    except json.JSONDecodeError:
        err = json.dumps({"error": "Geçersiz JSON"}, ensure_ascii=False)
        return 400, "application/json", err

    # a ve b anahtarlarını kontrol et
    if not all(k in data for k in ("a", "b")):
        err = json.dumps({"error": "JSON içinde 'a' ve 'b' anahtarları olmalı"}, ensure_ascii=False)
        return 400, "application/json", err

    # İşlemi yap
    try:
        a, b = float(data["a"]), float(data["b"])
        toplam = a + b
    except Exception:
        err = json.dumps({"error": "a ve b sayısal olmalı"}, ensure_ascii=False)
        return 400, "application/json", err

    # Sonucu döndür
    result = json.dumps({"a": a, "b": b, "sum": toplam}, ensure_ascii=False)
    return 200, "application/json", result
```


## Soru -2) Bu kod içerisinde ne yapılıyor ? Kodun amacı nedir?Kodda kullanılan teknikleri açıklayalım.

### 🧩 `handle_post_sum()` Fonksiyonunun Mantığı

Bu fonksiyon, **`POST /sum`** isteğini işleyen bir mini API endpoint’idir.  
Amacı: **İstemciden gelen JSON verisini okuyup, iki sayıyı toplayarak sonucu JSON olarak döndürmektir.**

Yani bu, senin API’ndeki **ilk “veri işleyen” fonksiyon**.  
FastAPI’deki şu fonksiyonun ilkel hali diyebiliriz:

```python
@app.post("/sum")
def sum_values(a: float, b: float):
    return {"sum": a + b}


## 🎯 Fonksiyonun Genel Akışı

* İstek türü kontrol edilir → Gövde gerçekten JSON mu?

* Gövde okunur ve çözümlenir → json.loads() ile Python sözlüğüne dönüştürülür.

* Verinin içeriği doğrulanır → “a” ve “b” anahtarları var mı?

* Matematiksel işlem yapılır → a + b

* Sonuç JSON olarak döndürülür.

* Hata durumunda (örneğin bozuk JSON, eksik veri, yanlış tip) → 400 Bad Request döner.

### ⚙️ Adım Adım Açıklama
#### 🔹 1️⃣ Content-Type kontrolü
```python
if headers.get("content-type", "") != "application/json":
```

* HTTP isteklerinde “Content-Type” başlığı, gövdenin formatını belirtir.
* Burada yalnızca application/json kabul edilir.

Yani istemci şunu göndermeli:

```bash
Content-Type: application/json
```
Aksi halde hata döndürülür:`

```json
{"error": "Content-Type application/json olmalı"}

### 🔹 2️⃣ Gövdeyi (body) JSON’a çevirme
```python
data = json.loads(body)
```

* body bir metin (örnek: '{"a": 10, "b": 5}')

>json.loads() → metni Python dict haline getirir:

{"a": 10, "b": 5}


* Eğer body geçersizse ({"a": 10, } gibi), json.JSONDecodeError hatası alınır ve fonksiyon:

> {"error": "Geçersiz JSON"}


döner.

### 🔹 3️⃣ Gerekli anahtarları kontrol etme
```python
if not all(k in data for k in ("a", "b")):
```


* Bu satır, hem "a" hem "b" anahtarlarının JSON içinde olup olmadığını kontrol eder.

Eksikse:

>{"error": "JSON içinde 'a' ve 'b' anahtarları olmalı"}

### 🔹 4️⃣ Sayısal değerleri dönüştürme ve toplama
```python
a, b = float(data["a"]), float(data["b"])
toplam = a + b
```


* Burada gelen değerleri sayıya çeviriyoruz (float) çünkü istemci "10" gibi string göndermiş olabilir.
* Eğer bu dönüştürme başarısız olursa (örneğin "a": "merhaba"), hata yakalanır:
```json
{"error": "a ve b sayısal olmalı"}
```
### 🔹 5️⃣ Başarılı sonuç döndürme
```python
result = json.dumps({"a": a, "b": b, "sum": toplam}, ensure_ascii=False)

```

* json.dumps() → Python sözlüğünü tekrar JSON metnine çevirir:

>{"a": 10.0, "b": 5.0, "sum": 15.0}


* ensure_ascii=False → Türkçe karakterler bozulmasın diye kullanılır.

Fonksiyon sonunda dönen değer:

>return 200, "application/json", result


Bu da:

>200 → HTTP durum kodu

"application/json" → içerik tipi

result → döndürülecek gövde (body)

### 🧠 Kullanılan Teknikler

| Teknik                         | Açıklama                                                     |
| ------------------------------ | ------------------------------------------------------------ |
| **Header kontrolü**            | `headers.get()` ile gelen isteğin formatını doğrulama        |
| **JSON Parsing**               | `json.loads()` ile metni Python nesnesine çevirme            |
| **Hata yakalama (try/except)** | JSON hatalarında veya tip hatalarında çökmeden hata döndürme |
| **Veri doğrulama**             | Gerekli anahtarların varlığını `all()` ile kontrol etme      |
| **Tip dönüştürme**             | `float()` ile string → sayı çevirme                          |
| **JSON oluşturma**             | `json.dumps()` ile Python dict → JSON string dönüşümü        |
| **HTTP kodları**               | 200 (başarılı), 400 (hatalı istek) kullanımı                 |


### 🧩 Örnek Akış

İstek:
```json
curl -X POST http://127.0.0.1:8080/sum \
     -H "Content-Type: application/json" \
     -d "{\"a\": 12, \"b\": 8}"
```


* Sunucu iç işlemi:

Header kontrolü ✅

JSON parse ✅

“a” ve “b” var ✅

Toplam hesaplandı ✅